In [1]:
import mysql.connector
import pandas as pd
from datetime import timedelta

# สร้างการเชื่อมต่อกับฐานข้อมูล
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',      # แทนที่ด้วยชื่อผู้ใช้ MySQL ของคุณ
        password='Root',  # แทนที่ด้วยรหัสผ่าน MySQL ของคุณ
        database='smart_farm'
    )
    cursor = connection.cursor()
    print("เชื่อมต่อฐานข้อมูลสำเร็จ")
except mysql.connector.Error as err:
    print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ: {err}")
    exit(1)

เชื่อมต่อฐานข้อมูลสำเร็จ


In [2]:
# อ่านข้อมูลจากไฟล์ CSV โดยใช้ pandas
df = pd.read_excel('data.xlsx')
# แปลงคอลัมน์ Timestamp เป็น datetime (pandas.Timestamp)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df.head()

,Timestamp,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,PH,Rainfall,Plant,Plant_season,...,Organic_matter,Irrigation_frequency (times/week),Crop_density,Pest_pressure,Fertilizer_usage,Growth_stage,Urban_area_proximity,Water_source_type,Frost_risk,Water_usage_efficiency
0,2025-01-01 08:00:00,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,September,...,3.121395,4,11.743910,57.607308,188.194958,Seedling,2.719614,Recycled,95.649985,1.193293
1,2025-01-01 09:00:00,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,September,...,2.142021,4,16.797101,74.736879,70.963629,Seedling,4.714427,Groundwater,77.265694,1.752672
2,2025-01-01 10:00:00,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,September,...,1.474974,1,12.654395,1.034478,191.976077,Seedling,30.431736,Groundwater,18.192168,3.035541
3,2025-01-01 11:00:00,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,June,...,8.393907,1,10.864360,24.091888,55.761388,Flowering,10.861071,Recycled,82.818720,1.273341
4,2025-01-01 12:00:00,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,March,...,5.202285,3,13.852910,38.811481,185.259702,Vegetative,47.190777,Recycled,25.466499,2.578671


In [ ]:
# วนลูปแต่ละแถวใน DataFrame
for index, row in df.iterrows():
    try:
        # ============================================================
        # 1. แทรกข้อมูลลงในตาราง Plant_Area
        # ============================================================
        plantation_area = row["Plantation_area"]
        soil_type = row["Soil_type"]
        
        query_pa = "SELECT PA_id FROM Plant_Area WHERE Plantation_area = %s AND Soil_type = %s"
        cursor.execute(query_pa, (plantation_area, soil_type))
        result = cursor.fetchone()
        if result:
            pa_id = result[0]
        else:
            insert_pa = "INSERT INTO Plant_Area (Plantation_area, Soil_type) VALUES (%s, %s)"
            cursor.execute(insert_pa, (plantation_area, soil_type))
            connection.commit()
            pa_id = cursor.lastrowid

        # ============================================================
        # 2. แทรกข้อมูลลงในตาราง Plant_Information
        # ============================================================
        plant = row["Plant"]
        urban_area_proximity = row["Urban_area_proximity"]
        insert_pi = "INSERT INTO Plant_Information (Plant, Urban_area_proximity, PA_id) VALUES (%s, %s, %s)"
        cursor.execute(insert_pi, (plant, urban_area_proximity, pa_id))
        connection.commit()
        plant_id = cursor.lastrowid

        # ============================================================
        # 3. แทรกข้อมูลลงในตาราง Sensor_Data
        # ============================================================
        # แปลง Timestamp เป็น Python datetime object
        original_dt = row["Timestamp"].to_pydatetime()
        nitrogen = row["Nitrogen"]
        phosphorus = row["Phosphorus"]
        potassium = row["Potassium"]
        temperature = row["Temperature"]
        humidity = row["Humidity"]
        ph_val = row["PH"]
        rainfall = row["Rainfall"]
        organic_matter = row["Organic_matter"]
        wind_speed = row["Wind_speed"]
        co2_concentration = row["CO2_concentration"]
        soil_moisture = row["Soil_moisture"]

        insert_sd = """
            INSERT INTO Sensor_Data 
            (TimestampPerHr, Temperature, humidity, ph, rainfall, nitrogen, phosphorus, potassium, organic_matter, Wind_speed, CO2_concentration, Soil_moisture, Plant_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_sd, (original_dt, temperature, humidity, ph_val, rainfall, nitrogen, phosphorus, potassium, organic_matter, wind_speed, co2_concentration, soil_moisture, plant_id))
        connection.commit()

        # ============================================================
        # 4. แทรกข้อมูลลงในตาราง Plant_Growth
        # ============================================================
        growth_stage = row["Growth_stage"]
        plant_season = row["Plant_season"]
        # แปลง Timestamp เป็นวันที่ 1 ของเดือน เวลา 00:00:00 (เดือนละครั้ง)
        dt_month = original_dt.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        insert_pg = "INSERT INTO Plant_Growth (Growth_stage, Plant_season, Timestamp_Per_Month, Plant_id) VALUES (%s, %s, %s, %s)"
        cursor.execute(insert_pg, (growth_stage, plant_season, dt_month, plant_id))
        connection.commit()

        # ============================================================
        # 5. แทรกข้อมูลลงในตาราง Crop_Management
        # ============================================================
        crop_density = row["Crop_density"]
        fertilizer_usage = row["Fertilizer_usage"]
        pest_pressure = row["Pest_pressure"]
        sunlight_exposure = row["Sunlight_exposure"]
        # แปลง Timestamp เป็นวันนั้น เวลา 00:00:00 (วันละครั้ง)
        dt_day = original_dt.replace(hour=0, minute=0, second=0, microsecond=0)
        insert_cm = "INSERT INTO Crop_Management (Crop_density, Fertilizer_usage, Pest_pressure, Sunlight_exposure, TimestampPerDay, Plant_id) VALUES (%s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_cm, (crop_density, fertilizer_usage, pest_pressure, sunlight_exposure, dt_day, plant_id))
        connection.commit()

        # ============================================================
        # 6. แทรกข้อมูลลงในตาราง Water_Management
        # ============================================================
        water_source_type = row["Water_source_type"]
        irrigation_frequency = row["Irrigation_frequency (times/week)"]
        water_usage_efficiency = row["Water_usage_efficiency"]
        frost_risk = row["Frost_risk"]
 
        days_to_sunday = 6 - dt_day.weekday()  
        dt_week = dt_day + timedelta(days=days_to_sunday)
        insert_wm = "INSERT INTO Water_Management (Water_source_type, Irrigation_frequency, Water_usage_efficiency, Frost_risk, TimestampPerWeek, Plant_id) VALUES (%s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_wm, (water_source_type, irrigation_frequency, water_usage_efficiency, frost_risk, dt_week, plant_id))
        connection.commit()

        print(f"Record {index} inserted successfully!")

    except Exception as e:
        print(f"Error at record {index}: {e}")
        connection.rollback()

Record 0 inserted successfully!
Record 1 inserted successfully!
Record 2 inserted successfully!
Record 3 inserted successfully!
Record 4 inserted successfully!
Record 5 inserted successfully!
Record 6 inserted successfully!
Record 7 inserted successfully!
Record 8 inserted successfully!
Record 9 inserted successfully!
Record 10 inserted successfully!
Record 11 inserted successfully!
Record 12 inserted successfully!
Record 13 inserted successfully!
Record 14 inserted successfully!
Record 15 inserted successfully!
Record 16 inserted successfully!
Record 17 inserted successfully!
Record 18 inserted successfully!
Record 19 inserted successfully!
Record 20 inserted successfully!
Record 21 inserted successfully!
Record 22 inserted successfully!
Record 23 inserted successfully!
Record 24 inserted successfully!
Record 25 inserted successfully!
Record 26 inserted successfully!
Record 27 inserted successfully!
Record 28 inserted successfully!
Record 29 inserted successfully!
Record 30 inserted s

In [4]:
# ปิดการเชื่อมต่อฐานข้อมูล
cursor.close()
connection.close()

In [27]:
import pandas as pd
from datetime import timedelta

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_excel('data.xlsx')

# แปลงคอลัมน์ Timestamp เป็น datetime object
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# สร้างคอลัมน์สำหรับ Plant_Growth:
# เปลี่ยน Timestamp ให้เป็นวันที่ 1 ของเดือน เวลา 00:00:00
df['Timestamp_Per_Month'] = df['Timestamp'].apply(lambda x: x.replace(day=1, hour=0, minute=0, second=0, microsecond=0))

# สร้างคอลัมน์สำหรับ Crop_Management:
# เปลี่ยน Timestamp ให้เป็นวันนั้น เวลา 00:00:00
df['TimestampPerDay'] = df['Timestamp'].apply(lambda x: x.replace(hour=0, minute=0, second=0, microsecond=0))

# สร้างคอลัมน์สำหรับ Water_Management:
# เปลี่ยน TimestampPerDay ให้เป็นวันอาทิตย์ของสัปดาห์นั้น เวลา 00:00:00
# ใน Python: weekday() ให้ Monday=0, Sunday=6
df['TimestampPerWeek'] = df['TimestampPerDay'].apply(lambda x: x + timedelta(days=(6 - x.weekday())))

# สร้าง DataFrame สำหรับแต่ละชุดข้อมูล
plant_growth_df = df[['Plant', 'Growth_stage', 'Plant_season', 'Timestamp_Per_Month']]
crop_management_df = df[['Plant', 'Crop_density', 'Fertilizer_usage', 'Pest_pressure', 'Sunlight_exposure', 'TimestampPerDay']]
water_management_df = df[['Plant', 'Water_source_type', 'Irrigation_frequency (times/week)', 'Water_usage_efficiency', 'Frost_risk', 'TimestampPerWeek']]

# บันทึก DataFrame แต่ละชุดลงไฟล์ CSV
plant_growth_df.to_csv('plant_growth.csv', index=False)
crop_management_df.to_csv('crop_management.csv', index=False)
water_management_df.to_csv('water_management.csv', index=False)

print("CSV files have been generated successfully.")


CSV files have been generated successfully.
